[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_21/iris_keras_neural_network.ipynb)

# Neuronale Netze mit TensorFlow/Keras: Iris-Datensatz 🌸🧠

**Ziel:** Moderne neuronale Netze mit TensorFlow/Keras für Iris-Klassifikation

In diesem Notebook lernen wir:
- **TensorFlow/Keras** für professionelle neuronale Netze
- **Validation Split** und echte Train/Validation Lernkurven
- **Callbacks** für Early Stopping und Model Checkpoints
- **Moderne Architekturen** mit Dropout und Batch Normalization
- **Tensorboard** Integration für Advanced Monitoring

**Vorteile gegenüber sklearn:**
- ✅ Echte Validierungs-Lernkurven
- ✅ GPU-Unterstützung
- ✅ Flexible Architekturen
- ✅ Production-Ready Models

## 1. Import Libraries und Setup

Zuerst laden wir TensorFlow/Keras und alle anderen benötigten Bibliotheken.

In [ ]:
# TensorFlow und Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

# Grundlegende Bibliotheken
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn für Daten und Metriken
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Warnings unterdrücken
import warnings
warnings.filterwarnings('ignore')

# Plotting Setup
plt.style.use('default')
sns.set_palette("husl")

# Reproduzierbarkeit - ALLE Seeds setzen!
import random
import os

# Python random
random.seed(42)
# NumPy random
np.random.seed(42)
# TensorFlow random
tf.random.set_seed(42)
# OS environment für TensorFlow
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED'] = '42'

# TensorFlow für deterministische Operationen konfigurieren
tf.config.experimental.enable_op_determinism()

print("🚀 TensorFlow/Keras Setup:")
print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print(f"GPU verfügbar: {len(tf.config.list_physical_devices('GPU')) > 0}")
print("🎲 Alle Random Seeds gesetzt für Reproduzierbarkeit!")
print("✅ Alle Bibliotheken erfolgreich importiert!")

## 2. Daten laden und vorbereiten

Wir laden die Iris-Daten und bereiten sie für Keras vor - inklusive One-Hot Encoding für die Labels.

In [ ]:
# Iris-Datensatz laden
iris = load_iris()
X, y = iris.data, iris.target

print("📊 Iris-Datensatz:")
print(f"Features: {X.shape}")
print(f"Labels: {y.shape}")
print(f"Klassen: {iris.target_names}")
print(f"Feature-Namen: {iris.feature_names}")

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n📊 Datenaufteilung:")
print(f"Training: {X_train.shape[0]} Samples")
print(f"Test: {X_test.shape[0]} Samples")

# Feature Scaling für neuronale Netze
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# One-Hot Encoding für Labels (wichtig für Keras!)
y_train_onehot = to_categorical(y_train, num_classes=3)
y_test_onehot = to_categorical(y_test, num_classes=3)

print(f"\n🔄 Preprocessing:")
print(f"Features skaliert: Mean={X_train_scaled.mean():.3f}, Std={X_train_scaled.std():.3f}")
print(f"Labels Shape: {y_train.shape} → {y_train_onehot.shape}")
print(f"One-Hot Beispiel: {y_train[0]} → {y_train_onehot[0]}")

# Klassenverteilung prüfen
unique, counts = np.unique(y_train, return_counts=True)
print(f"\n📈 Klassenverteilung (Training):")
for cls, count in zip(iris.target_names, counts):
    print(f"  {cls}: {count} Samples")

## 3. Keras Modell erstellen

Jetzt bauen wir ein modernes neuronales Netz mit Keras - flexibler und mächtiger als sklearn!

In [ ]:
# Modell mit Sequential API erstellen
model = keras.Sequential([
    # Input Layer - explizit definiert
    layers.Input(shape=(4,), name='features'),
    
    # Hidden Layer 1 mit Batch Normalization
    layers.Dense(64, activation='relu', name='hidden1'),
    layers.BatchNormalization(name='bn1'),
    layers.Dropout(0.3, name='dropout1'),
    
    # Hidden Layer 2
    layers.Dense(32, activation='relu', name='hidden2'),
    layers.BatchNormalization(name='bn2'),
    layers.Dropout(0.2, name='dropout2'),
    
    # Hidden Layer 3 (optional - experimentiere!)
    layers.Dense(16, activation='relu', name='hidden3'),
    
    # Output Layer - Softmax für Multi-Class
    layers.Dense(3, activation='softmax', name='output')
], name='IrisClassifier')

# Modell-Architektur anzeigen
print("🧠 Neuronales Netz Architektur:")
model.summary()

# Modell kompilieren
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',  # Für One-Hot Labels
    metrics=['accuracy', 'categorical_crossentropy']
)

print("\n⚙️ Kompilierung abgeschlossen:")
print(f"Optimizer: Adam (lr=0.001)")
print(f"Loss: Categorical Crossentropy")
print(f"Metriken: Accuracy + Loss")

# Parameter zählen
total_params = model.count_params()
print(f"\n📊 Model Statistiken:")
print(f"Trainierbare Parameter: {total_params:,}")

## 4. Training mit Validation Split und Callbacks

Hier kommt der große Vorteil von Keras: Echte Validierungs-Lernkurven und moderne Callbacks!

In [ ]:
# Callbacks für smartes Training
callbacks = [
    # Early Stopping - stoppt bei Overfitting
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Learning Rate Scheduler
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=0.0001,
        verbose=1
    )
]

print("🎯 Callbacks konfiguriert:")
print("✅ Early Stopping (patience=20)")
print("✅ Learning Rate Reduction")

# Training mit Validation Split
print("\n🚀 Training gestartet...")
history = model.fit(
    X_train_scaled, y_train_onehot,
    validation_split=0.2,  # 20% für Validation - DAS ist der Schlüssel!
    epochs=200,
    batch_size=16,
    callbacks=callbacks,
    verbose=1,  # Fortschritt anzeigen
    shuffle=True,  # Explizit für Reproduzierbarkeit
    validation_batch_size=16  # Validation Batch Size für Konsistenz
)

print(f"\n✅ Training beendet nach {len(history.history['loss'])} Epochen")

## 5. Training- und Validierungs-Lernkurven visualisieren

Endlich echte Train/Validation Curves - das ging mit sklearn nicht!

In [ ]:
# Training History extrahieren
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(1, len(train_loss) + 1)

# Subplot für Loss und Accuracy
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Loss Plot
ax1.plot(epochs, train_loss, 'b-', label='Training Loss', linewidth=2)
ax1.plot(epochs, val_loss, 'r-', label='Validation Loss', linewidth=2)
ax1.set_title('Training & Validation Loss', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy Plot
ax2.plot(epochs, train_acc, 'b-', label='Training Accuracy', linewidth=2)
ax2.plot(epochs, val_acc, 'r-', label='Validation Accuracy', linewidth=2)
ax2.set_title('Training & Validation Accuracy', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Training-Statistiken
min_val_loss = min(val_loss)
max_val_acc = max(val_acc)
final_train_loss = train_loss[-1]
final_val_loss = val_loss[-1]

print("📊 Training-Ergebnisse:")
print(f"Beste Validation Loss: {min_val_loss:.4f}")
print(f"Beste Validation Accuracy: {max_val_acc:.4f} ({max_val_acc*100:.2f}%)")
print(f"Final Training Loss: {final_train_loss:.4f}")
print(f"Final Validation Loss: {final_val_loss:.4f}")

# Overfitting Check
if final_train_loss < final_val_loss * 0.7:
    print("⚠️  Mögliches Overfitting erkannt!")
elif abs(final_train_loss - final_val_loss) < 0.1:
    print("✅ Gute Generalisierung!")
else:
    print("👍 Moderate Generalisierung")

## 6. Model Evaluation auf Test Set

Jetzt testen wir das trainierte Modell auf dem echten Test Set.

In [ ]:
# Evaluation auf Test Set
test_loss, test_accuracy, test_cat_crossentropy = model.evaluate(
    X_test_scaled, y_test_onehot, verbose=0
)

print("🎯 Test Set Performance:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Vorhersagen machen
y_pred_proba = model.predict(X_test_scaled, verbose=0)
y_pred = np.argmax(y_pred_proba, axis=1)

# Classification Report
print("\n📊 Detaillierte Klassifikation:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

# Konfusionsmatrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.title(f'Confusion Matrix\nTest Accuracy: {test_accuracy:.3f}', fontweight='bold')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Fehleranalyse
errors = np.where(y_test != y_pred)[0]
if len(errors) > 0:
    print(f"\n🔍 Fehleranalyse: {len(errors)} Fehler von {len(y_test)} Samples")
    for i in errors:
        actual = iris.target_names[y_test[i]]
        predicted = iris.target_names[y_pred[i]]
        confidence = np.max(y_pred_proba[i])
        print(f"Sample {i}: {actual} → {predicted} (Confidence: {confidence:.3f})")
else:
    print("\n🎉 Perfekte Klassifikation! Keine Fehler im Test Set.")

## 7. Vorhersagen mit Confidence-Analyse

Keras gibt uns detaillierte Wahrscheinlichkeiten für jede Klasse - perfekt für Unsicherheits-Analyse!

In [ ]:
# Neue Beispiel-Blüten für Vorhersagen
new_flowers = np.array([
    [5.1, 3.5, 1.4, 0.2],  # Typische Setosa
    [6.9, 3.1, 4.9, 1.5],  # Typische Versicolor
    [6.3, 3.3, 6.0, 2.5],  # Typische Virginica
    [5.8, 2.7, 4.1, 1.0],  # Grenzfall
    [4.5, 2.0, 3.5, 1.0]   # Schwieriger Fall
])

# Skalierung anwenden
new_flowers_scaled = scaler.transform(new_flowers)

# Vorhersagen mit Keras
predictions_proba = model.predict(new_flowers_scaled, verbose=0)
predictions = np.argmax(predictions_proba, axis=1)

print("🔮 Keras Vorhersagen für neue Iris-Blüten:")
print("=" * 70)

feature_names = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']

for i, (flower, pred, probs) in enumerate(zip(new_flowers, predictions, predictions_proba)):
    print(f"\n🌸 Blüte {i+1}:")
    print(f"Features: {dict(zip(feature_names, flower))}")
    print(f"Vorhersage: {iris.target_names[pred]} (Klasse {pred})")
    
    print("Wahrscheinlichkeiten:")
    for j, (species, prob) in enumerate(zip(iris.target_names, probs)):
        bar = "█" * int(prob * 20)  # Visual bar
        print(f"  {species:10}: {prob:.4f} ({prob*100:5.1f}%) {bar}")
    
    # Confidence Analysis
    max_prob = np.max(probs)
    entropy = -np.sum(probs * np.log(probs + 1e-10))  # Uncertainty measure
    
    if max_prob > 0.95:
        confidence = "Sehr sicher 🎯"
    elif max_prob > 0.8:
        confidence = "Sicher 👍"
    elif max_prob > 0.6:
        confidence = "Moderate Sicherheit 🤔"
    else:
        confidence = "Unsicher ❓"
    
    print(f"Confidence: {confidence}")
    print(f"Max Probability: {max_prob:.4f}")
    print(f"Entropy (Unsicherheit): {entropy:.4f}")
    print("-" * 50)

In [ ]:
# Wahrscheinlichkeiten visualisieren
plt.figure(figsize=(15, 10))

for i in range(len(new_flowers)):
    plt.subplot(2, 3, i+1)
    
    # Balkendiagramm
    bars = plt.bar(iris.target_names, predictions_proba[i], 
                  color=['red', 'green', 'blue'], alpha=0.7)
    
    # Vorhersage hervorheben
    predicted_idx = predictions[i]
    bars[predicted_idx].set_alpha(1.0)
    bars[predicted_idx].set_edgecolor('black')
    bars[predicted_idx].set_linewidth(3)
    
    plt.title(f'Blüte {i+1}: {iris.target_names[predicted_idx]}', 
             fontweight='bold', fontsize=12)
    plt.ylabel('Wahrscheinlichkeit')
    plt.ylim(0, 1)
    
    # Werte anzeigen
    for bar, prob in zip(bars, predictions_proba[i]):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{prob:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # Entropy als Unsicherheitsmaß
    entropy = -np.sum(predictions_proba[i] * np.log(predictions_proba[i] + 1e-10))
    plt.text(0.5, 0.85, f'Entropy: {entropy:.3f}', transform=plt.gca().transAxes,
            ha='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

# Model-Interpretabilität
print("\n🧠 Model-Insights:")
print(f"📊 Trainierbare Parameter: {model.count_params():,}")

# Architektur korrekt extrahieren
architecture = []
for layer in model.layers:
    if hasattr(layer, 'units'):  # Dense layers
        architecture.append(layer.units)
print(f"🏗️  Architektur: {architecture}")

print(f"⚙️  Optimizer: Adam")
print(f"🎯 Best Val Accuracy: {max(val_acc):.4f}")
print(f"🔥 Final Test Accuracy: {test_accuracy:.4f}")

# Gewichte der ersten Dense Schicht analysieren (Feature Importance grob)
# Die erste Dense Schicht ist bei Index 1 (nach Input Layer)
first_dense_layer = None
for layer in model.layers:
    if isinstance(layer, layers.Dense):
        first_dense_layer = layer
        break

if first_dense_layer is not None:
    first_layer_weights = first_dense_layer.get_weights()[0]  # Dense layer weights
    feature_importance = np.mean(np.abs(first_layer_weights), axis=1)
    
    print(f"\n🔍 Feature Importance (erste Dense Schicht):")
    for i, (feature, importance) in enumerate(zip(iris.feature_names, feature_importance)):
        print(f"  {feature}: {importance:.3f}")
else:
    print(f"\n🔍 Keine Dense Schicht für Feature Importance gefunden.")

## 8. Model-Persistierung und Export

Ein großer Vorteil von Keras: Einfaches Speichern und Laden von Modellen!

In [ ]:
# Model und Scaler speichern
model_path = "iris_keras_model.h5"
scaler_path = "iris_scaler.pkl"

# Keras Model speichern
model.save(model_path)
print(f"✅ Model gespeichert: {model_path}")

# Scaler mit pickle speichern
import pickle
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"✅ Scaler gespeichert: {scaler_path}")

# Model laden und testen (Beispiel)
print("\n🔄 Model-Loading Test:")
loaded_model = keras.models.load_model(model_path)
with open(scaler_path, 'rb') as f:
    loaded_scaler = pickle.load(f)

# Test mit geladenem Model
test_sample = np.array([[5.0, 3.0, 1.5, 0.3]])
test_sample_scaled = loaded_scaler.transform(test_sample)
prediction = loaded_model.predict(test_sample_scaled, verbose=0)

print(f"Test-Vorhersage: {iris.target_names[np.argmax(prediction)]}")
print(f"Wahrscheinlichkeiten: {prediction[0]}")
print("✅ Model erfolgreich geladen und getestet!")

# Model-Export Informationen
print(f"\n📦 Export-Informationen:")
print(f"Model-Datei: {model_path}")
print(f"Scaler-Datei: {scaler_path}")
print(f"TensorFlow Version: {tf.__version__}")
print(f"Model Input Shape: {model.input_shape}")
print(f"Model Output Shape: {model.output_shape}")

# Production-Ready Code Snippet
print(f"\n💻 Production Code Snippet:")
print("""
# Laden und Verwenden des Modells:
import tensorflow as tf
import pickle
import numpy as np

# Model und Scaler laden
model = tf.keras.models.load_model('iris_keras_model.h5')
with open('iris_scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Vorhersage für neue Daten
def predict_iris(sepal_length, sepal_width, petal_length, petal_width):
    features = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
    features_scaled = scaler.transform(features)
    prediction = model.predict(features_scaled)
    species = ['setosa', 'versicolor', 'virginica']
    return species[np.argmax(prediction)], prediction[0]

# Beispiel-Verwendung
species, probabilities = predict_iris(5.1, 3.5, 1.4, 0.2)
print(f"Vorhersage: {species}")
print(f"Wahrscheinlichkeiten: {probabilities}")
""")

## 🎯 Zusammenfassung: Keras vs. Sklearn

**Was haben wir mit TensorFlow/Keras gewonnen?**

### ✅ **Keras Vorteile:**

**1. Echte Validation Split:**
- `validation_split=0.2` erzeugt automatisch Train/Val Aufteilung
- Separate Lernkurven für Training und Validation
- Overfitting-Detection in Echtzeit

**2. Moderne Callbacks:**
- **Early Stopping:** Automatisches Stoppen bei Overfitting
- **Learning Rate Scheduling:** Adaptive Lernrate
- **Model Checkpoints:** Beste Gewichte speichern

**3. Flexible Architektur:**
- **Batch Normalization:** Stabileres Training
- **Dropout:** Regularisierung gegen Overfitting
- **Multiple Hidden Layers:** Tiefere Netze möglich

**4. Production Features:**
- **Model Saving/Loading:** .h5 Format für Deployment
- **GPU-Unterstützung:** Automatische GPU-Nutzung
- **TensorBoard Integration:** Advanced Monitoring

**5. Professionelle Metriken:**
- **Entropy-basierte Unsicherheit:** Bessere Confidence-Analyse
- **Detailed Probabilities:** Feinere Vorhersage-Kontrolle
- **Feature Importance:** Erste Schicht Gewichte analysieren

### 📊 **Vergleich der Ergebnisse:**

| Aspekt | Sklearn MLPClassifier | Keras/TensorFlow |
|--------|----------------------|------------------|
| **Validation Curves** | ❌ Nur Training Loss | ✅ Train + Validation |
| **Early Stopping** | ❌ Nur max_iter | ✅ Intelligentes Stoppen |
| **Architecture** | 🟡 Begrenzt | ✅ Vollständig flexibel |
| **Regularization** | 🟡 Nur L2 | ✅ Dropout + Batch Norm |
| **Model Export** | ❌ Pickle only | ✅ .h5 + Production Ready |
| **GPU Support** | ❌ Nein | ✅ Automatisch |
| **Monitoring** | 🟡 Basic | ✅ TensorBoard + Callbacks |

### 🚀 **Wann welches Framework?**

**Sklearn MLPClassifier für:**
- 🎯 Schnelle Prototypen und Experimente
- 📚 Lernzwecke und einfache Demos
- 🔬 Kleine Datensätze (<10k Samples)
- ⚡ Wenn Training-Zeit unwichtig ist

**TensorFlow/Keras für:**
- 🏭 Production-Systeme
- 📈 Große Datensätze (>100k Samples)
- 🧠 Komplexe Architekturen (CNNs, RNNs, etc.)
- 🎮 GPU-beschleunigtes Training
- 📊 Detailliertes Monitoring und Debugging

### 💡 **Haupterkenntnisse:**

1. **Keras gibt dir Kontrolle:** Validation Split, Callbacks, flexible Architekturen
2. **Besseres Monitoring:** Train/Val Curves zeigen Overfitting sofort
3. **Production-Ready:** .h5 Models sind deployment-fähig
4. **Moderne Features:** Batch Normalization und Dropout out-of-the-box
5. **Skalierbarkeit:** Von Iris (150 Samples) bis ImageNet (Millionen)

**Fazit:** Für ernsthafte Deep Learning Projekte führt kein Weg an TensorFlow/Keras vorbei! 🚀🧠